<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/new_item_cpd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ruptures

In [ ]:
import pandas as pd
import numpy as np
import datetime
import ruptures as rpt
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('cpd_1.csv')
df.columns = [col.lower() for col in df.columns]
df['txn_dte'] = pd.to_datetime(df['txn_dte'])
df.set_index('txn_dte', inplace = True)
#Convert the time series values to a numpy 1D array
points = np.array(df['gross_amt'])
df

In [ ]:
df.columns

In [ ]:
df['gross_amt'].plot()

In [ ]:
df.index

In [ ]:
df_agg = df.reset_index().groupby(['store_id', 'consumer_selling_cd', 'txn_dte']).agg({'gross_amt' : np.sum}).reset_index()
df_agg['txn_dte'] = df_agg['txn_dte'].dt.strftime('%Y-%m-%d')
df_agg

In [ ]:
from datetime import datetime
from datetime import timedelta
max_date = datetime.strptime(df_agg.txn_dte.max(), '%Y-%m-%d')
min_date = datetime.strptime(df_agg.txn_dte.min(), '%Y-%m-%d')
date_range = pd.date_range(max_date - timedelta(days=365), max_date, freq = 'd')
date_range

In [ ]:
driver_df = pd.DataFrame()
temp_list = []
for dt in date_range:
    for e in df_agg[['store_id', 'consumer_selling_cd', 'txn_dte']].drop_duplicates().values.tolist():
        temp_list.append((e[0], e[1], dt.strftime("%Y-%m-%d")))
driver_df = pd.DataFrame(temp_list, columns = ['store_id', 'consumer_selling_cd', 'txn_dte'])
driver_df

In [ ]:
df_agg = pd.merge(driver_df, df_agg, how = 'left', on = ['store_id', 'consumer_selling_cd', 'txn_dte']).fillna(0).set_index('txn_dte')
points = np.array(df_agg['gross_amt'])

In [ ]:
df_agg['gross_amt'].plot(figsize=(20,5))

In [ ]:
#RUPTURES PACKAGE
#Changepoint detection with the Pelt search method
model_rbf="rbf"
algo_pelt = rpt.Pelt(model=model_rbf).fit(points)
result_pelt = algo_pelt.predict(pen=10)
rpt.display(points, result_pelt, figsize=(10, 6))
plt.title('Change Point Detection: Pelt Search Method')
plt.show()

In [ ]:
#Changepoint detection with the Binary Segmentation search method
model_bin_seg = "l2"  
algo_bin_seg = rpt.Binseg(model=model_bin_seg).fit(points)
result_bin_seg = algo_bin_seg.predict(n_bkps=1)
# show results
rpt.show.display(points, result_bin_seg, figsize=(10, 6))
plt.title('Change Point Detection: Binary Segmentation Search Method')
plt.show()

In [ ]:
#Changepoint detection with window-based search method
model_wb = "l2"  
algo_wb = rpt.Window(width=40, model=model_wb).fit(points)
model_wb = algo_wb.predict(n_bkps=1)
rpt.show.display(points, model_wb, figsize=(10, 6))
plt.title('Change Point Detection: Window-Based Search Method')
plt.show()

In [ ]:
#Changepoint detection with dynamic programming search method
model_dyn_prog = "l1"  
algo_dyn_prog = rpt.Dynp(model=model_dyn_prog, min_size=3, jump=5).fit(points)
res_dyn_prog = algo_dyn_prog.predict(n_bkps=1)
rpt.show.display(points, res_dyn_prog, figsize=(10, 6))
plt.title('Change Point Detection: Dynamic Programming Search Method')
plt.show()